In [ ]:
%cd ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych

In [ ]:
%matplotlib inline

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
CONDITIONS = ["go_gloexp", "lo_gloexp", "go_seqctl", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [ ]:
def hippocampal_channels(channels):
    for l in channels.location.values:
        yield ("DG-" in l or "CA" in l)

def visual_channels(channels):
    for l in channels.location.values:
        yield ("VIS" in l)

def subcortical_channels(channels):
    for l in channels.location.values:
        yield ("DG-" in l or "CA" in l or "MB" in l or "SCi" in l or "POST" in l)

In [ ]:
NWB_SUBJECTS = glob.glob('/mnt/data/000253/sub-*/')

In [ ]:
PILOT_FILES = []
REQUIRED_PROBES = 6

In [ ]:
evoked = {}

In [ ]:
for cond in CONDITIONS:
    for s, subject_dir in enumerate(sorted(NWB_SUBJECTS)):
        subject = subject_dir.split('/')[-2]
        if not os.path.exists(subject_dir + "/" + cond):
            continue
        sampling = epych.recording.Sampling.unpickle(subject_dir + "/" + cond).smap(lambda sig: sig.median_filter())
        evoked[(cond, subject, True)] = sampling.baseline_correct(0, PRETRIAL_SECONDS).erp()
        del sampling
        logging.info("Loaded LFPs for %s in subject %s" % (cond, subject))

In [ ]:
def common_area_title(signame, signal):
    prefix = os.path.commonprefix(list(signal.channels.location.values))
    return "%s (%s)" % (prefix if prefix else "Subcortical", signame)

In [ ]:
for cond in CONDITIONS:
    logging.info("Plotting baseline-corrected LFPs for %s across subjects" % cond)
    for s, subject_dir in enumerate(sorted(NWB_SUBJECTS)):
        subject = subject_dir.split('/')[-2]
        if (cond, subject, True) not in evoked:
            continue
        erp = evoked[(cond, subject, True)].baseline_correct(0., POSTTRIAL_SECONDS)
        events = {
            "Oddball Onset": (erp.trials["stim3_start"].mean(), 'lightgreen'),
            "Oddball Offset": (erp.trials["stim3_end"].mean(), 'red'),
        }
        cortical = erp.smap(lambda sig: sig.select_channels(list(visual_channels(sig.channels))))
        cortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_visual_lfp.pdf" % (subject, cond), sigtitle=common_area_title, **events)

        subcortical = erp.smap(lambda sig: sig.select_channels(list(subcortical_channels(sig.channels))))
        subcortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_subcortical_lfp.pdf" % (subject, cond), sigtitle=common_area_title, **events)

In [ ]:
for cond in CONDITIONS:
    logging.info("Plotting baseline-corrected CSDs for %s across subjects" % cond)
    for s, subject_dir in enumerate(sorted(NWB_SUBJECTS)):
        subject = subject_dir.split('/')[-2]
        if (cond, subject, True) not in evoked:
            continue
        csd = evoked[(cond, subject, True)].smap(lambda sig: sig.downsample(4).current_source_density(depth_column="vertical"))
        events = {
            "Oddball Onset": (erp.trials["stim3_start"].mean(), 'lightgreen'),
            "Oddball Offset": (erp.trials["stim3_end"].mean(), 'red'),
        }
        cortical = csd.smap(lambda sig: sig.select_channels(list(visual_channels(sig.channels))))
        cortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_visual_csd.pdf" % (subject, cond), sigtitle=common_area_title, cmap='jet_r', **events)

        subcortical = csd.smap(lambda sig: sig.select_channels(list(subcortical_channels(sig.channels))))
        subcortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_subcortical_csd.pdf" % (subject, cond), sigtitle=common_area_title, cmap='jet_r', **events)